# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
from nltk.metrics.distance import edit_distance
import pandas as pd

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
!pip install pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [ ]:
word = "велечайшим"
with open ("litw-win.txt", "r", encoding='windows-1251') as fp:
    words = [line.strip().split()[-1] for line in fp]
words[-5:]

['высокопревосходительства',
 'попреблагорассмотрительст',
 'попреблагорассмотрительствующемуся',
 'убегающих',
 'уменьшившейся']

In [ ]:
min(words, key=lambda w: edit_distance(w, word))

'величайшим'

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [ ]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
stemmer = SnowballStemmer('russian')
for word in word_tokenize(text):
    result = stemmer.stem(word)
    print(result)

с
велечайш
усил
выбра
из
поток
убега
люд
кутуз
со
свит
уменьшевш
вдво
поеха
на
звук
выстрел
русск
оруд


In [ ]:
morph = pymorphy2.MorphAnalyzer()
for word in word_tokenize(text):
    result = morph.parse(word)[0].normalized.word
    print(result)

с
велечайший
усилие
выбраться
из
поток
убегать
человек
кутузов
с
свита
уменьшевшийся
вдвое
поехать
на
звук
выстрел
русский
орудие


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [ ]:
text = '''Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. '''
sents = sent_tokenize(text)
sents

['Считайте слова из файла `litw-win.txt` и запишите их в список `words`.',
 'В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`.',
 'Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.']

In [ ]:
cv = CountVectorizer()
cv.fit(sents)
sents_cv = cv.transform(sents).toarray()
sents_cv

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]])

In [ ]:
cv.vocabulary_

{'считайте': 32,
 'слова': 24,
 'из': 12,
 'файла': 33,
 'litw': 0,
 'win': 2,
 'txt': 1,
 'запишите': 11,
 'их': 14,
 'список': 31,
 'words': 3,
 'заданном': 9,
 'предложении': 22,
 'исправьте': 13,
 'все': 5,
 'опечатки': 21,
 'заменив': 10,
 'опечатками': 20,
 'на': 16,
 'ближайшие': 4,
 'смысле': 27,
 'расстояния': 23,
 'левенштейна': 15,
 'ним': 18,
 'списка': 29,
 'что': 34,
 'слове': 25,
 'есть': 8,
 'опечатка': 19,
 'если': 7,
 'данное': 6,
 'слово': 26,
 'не': 17,
 'содержится': 28,
 'списке': 30}

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize

preprocessed_descriptions = pd.read_csv("ssh.csv")

words_set = set()
words_list = list()
words = [word_tokenize(item) for item in preprocessed_descriptions["preprocessed_descriptions"].to_list() if isinstance(item, str)]

[[words_set.add(x) for x in item] for item in words]
[[words_list.append(x) for x in item] for item in words]

for item in words:
    print(item)

Выходные данные были обрезаны до нескольких последних строк (5000).
['this', 'one', 'comes', 'from', 'my', 'newlywed', 'days', 'when', 'i', 'received', 'a', 'whole', 'set', 'of', 'illustrated', 'recipe', 'cards', 'i', 'tweaked', 'it', 'to', 'suit', 'our', 'tastes', 'into', 'a', 'very', 'flavorsome', 'and', 'aromatic', 'meal', 'the', 'rice', 'cooks', 'in', 'the', 'microwave', 'while', 'the', 'chicken', 'works', 'in', 'the', 'frying', 'pan', 'whydinner', 'will', 'be', 'ready', 'before', 'you', 'know', 'italtho', 'you', 'will', 'have', 'to', 'listen', 'to', 'lots', 'of', 'begging', 'as', 'in', 'what', 'are', 'you', 'making', 'that', 'smells', 'so', 'good', 'and', 'are', 'we', 'ever', 'going', 'to', 'eat']
['i', 'saw', 'ina', 'garten', 'prepare', 'this', 'on', 'the', 'show', 'where', 'she', 'served', 'this', 'at', 'a', 'beach', 'volleball', 'picnic', 'this', 'combination', 'sounded', 'wondeful', 'i', 'didnt', 'allow', 'for', 'the', 'cooling', 'time', 'of', 'several', 'hours', 'update', '03

In [ ]:
print(f"Весего {len(words_list)} слов\nСреди них {len(words_set)} уникальных")

Весего 1069254 слов
Среди них 32868 уникальных


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
import random
data = random.sample(list(words_set), 10)
for i in range(0,len(data),2):
    x, y = data[i], data[i+1]
    print(edit_distance(x, y), x, y)

12 buttersugaroat cinnamony
8 mexico recreation
8 beefstew 440g
7 i indicate
6 appetite necessity


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [ ]:
from typing import Set, List

def same_words(word: str, k: int, words_data: Set[str]):
    buf_tuple = [(edit_distance(word, item), item) for item in words_data]
    buf_tuple.sort(key = lambda x: x[0])
    return buf_tuple[:k]

same_words("sadness", 10, words_set)

[(2, 'sydneys'),
 (3, 'guiness'),
 (3, 'annss'),
 (3, 'saves'),
 (3, 'weakness'),
 (3, 'dress'),
 (3, 'stainless'),
 (3, 'kindness'),
 (3, 'nanners'),
 (3, 'painless')]

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [ ]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

words_df = pd.DataFrame(words_set)
words_df.columns = ['word']
words_df['stemmed_word'] = words_df.apply(lambda x: stemmer.stem(x["word"]), axis=1)
words_df['normalized_word'] = words_df.apply(lambda x: lemmatizer.lemmatize(x["word"], "v"), axis=1)
words_df[(words_df["word"] != words_df["normalized_word"]) & (words_df["stemmed_word"] != words_df["normalized_word"])]

,word,stemmed_word,normalized_word
13,hung,hung,hang
50,assures,assur,assure
60,photocopied,photocopi,photocopy
69,pictures,pictur,picture
90,relies,reli,rely
...,...,...,...
32688,announces,announc,announce
32698,replicating,replic,replicate
32739,strewn,strewn,strew
32793,conditions,condit,condition


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))

words_filtered = [item for item in words_list if item not in stopwords_set]
diff = round(len(words_filtered)/len(words_list)*100,2)
print(f"Всего слов: {len(words_list)}\nС удалением стоп-слов: {len(words_filtered)}\nДоля стоп-слов: {diff}%")

Всего слов: 1069254
С удалением стоп-слов: 580889
Доля стоп-слов: 54.33%


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

preprocessed_descriptions = pd.read_csv("ssh.csv")
data = preprocessed_descriptions.sample(5)
data

,Unnamed: 0,name,preprocessed_descriptions
4616,4616,caprese pasta salad,this is a great little side dish for a bbq and...
291,291,aarti s creamy pistachio pops,recipe courtesy aarti sequeira
9987,9987,easy delicious tilapia,i had a half of a cup of finely chopped green ...
3726,3726,breakfast skillet,inspired by a dish i enjoy at a restaurant you...
5306,5306,cheesy red bliss garlic potatoes,so simple so delicious so inexpensive you can ...


In [ ]:
vectorizer = TfidfVectorizer(analyzer="word", stop_words="english")
vectorizer.fit(data["preprocessed_descriptions"])

def vectorizer_processing(x):
    sents = [x["preprocessed_descriptions"]]
    vector = vectorizer.transform(sents)
    return vector.toarray()

data['TfidfVectorizer'] = data.apply(lambda x: vectorizer_processing(x), axis=1)

In [ ]:
valuess = [(k,v) for (k, v) in dict(vectorizer.vocabulary_).items()]
valuess.sort(key=lambda x: x[1])
valuess

[('aarti', 0),
 ('achieve', 1),
 ('add', 2),
 ('added', 3),
 ('bacon', 4),
 ('balanced', 5),
 ('bbq', 6),
 ('cheese', 7),
 ('chicken', 8),
 ('cholesterol', 9),
 ('chopped', 10),
 ('cooked', 11),
 ('courtesy', 12),
 ('crazy', 13),
 ('crumbled', 14),
 ('cup', 15),
 ('delicious', 16),
 ('did', 17),
 ('dinner', 18),
 ('dish', 19),
 ('dont', 20),
 ('endless', 21),
 ('enjoy', 22),
 ('fats', 23),
 ('finely', 24),
 ('finished', 25),
 ('good', 26),
 ('great', 27),
 ('green', 28),
 ('grilled', 29),
 ('half', 30),
 ('healthier', 31),
 ('help', 32),
 ('hot', 33),
 ('id', 34),
 ('inexpensive', 35),
 ('inspired', 36),
 ('ive', 37),
 ('left', 38),
 ('levels', 39),
 ('like', 40),
 ('little', 41),
 ('macaroni', 42),
 ('mayobased', 43),
 ('monounsaturated', 44),
 ('oil', 45),
 ('olive', 46),
 ('olives', 47),
 ('onions', 48),
 ('packed', 49),
 ('people', 50),
 ('pepper', 51),
 ('pretty', 52),
 ('quick', 53),
 ('recipe', 54),
 ('red', 55),
 ('restaurant', 56),
 ('room', 57),
 ('salad', 58),
 ('sausage', 5

In [ ]:
for word, vector in zip(data["preprocessed_descriptions"].to_list(), data["TfidfVectorizer"].to_list()):
    print(vector.shape)
    print(f"{word}\n{vector}\n{'-'*10}\n")

(1, 80)
this is a great little side dish for a bbq and much healthier than the typical mayobased macaroni salad olive oil is packed with monounsaturated good fats and can help to achieve balanced cholesterol levels i usually serve this at room temperature but ive also served it hot and topped it with grilled chicken for a quick weeknight dinner
[[0.         0.17594885 0.         0.         0.         0.17594885
  0.17594885 0.         0.17594885 0.17594885 0.         0.
  0.         0.         0.         0.         0.         0.
  0.17594885 0.14195442 0.         0.         0.         0.17594885
  0.         0.         0.17594885 0.17594885 0.         0.17594885
  0.         0.17594885 0.17594885 0.17594885 0.         0.
  0.         0.14195442 0.         0.17594885 0.         0.17594885
  0.17594885 0.17594885 0.17594885 0.17594885 0.17594885 0.
  0.         0.17594885 0.         0.         0.         0.17594885
  0.         0.         0.         0.17594885 0.17594885 0.
  0.         

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [ ]:
import scipy
import numpy as np
import itertools

In [ ]:
max_pair = None
max_result = -1

In [ ]:
coeff_dict = {}
vectorizer3 = TfidfVectorizer(analyzer="word", stop_words="english")
transform3 = vectorizer3.fit_transform(data["preprocessed_descriptions"].to_list())

all_data = list(zip(data["preprocessed_descriptions"].to_list(), transform3.toarray()))

for pair in itertools.product(all_data, repeat=2):
    
    text1, matrix1 = pair[0]
    text2, matrix2 = pair[1]
    result = scipy.spatial.distance.cosine(matrix1, matrix2)
    inverse_result = 1 - result
    
    if text1 not in coeff_dict:
        coeff_dict[text1] = []
    coeff_dict[text1].append(inverse_result)
    

    if inverse_result > max_result and text1 != text2:
        max_result = inverse_result
        max_pair = (text1, text2)
    
    print(f"{text1}\n{text2}\n{inverse_result}\n")

this is a great little side dish for a bbq and much healthier than the typical mayobased macaroni salad olive oil is packed with monounsaturated good fats and can help to achieve balanced cholesterol levels i usually serve this at room temperature but ive also served it hot and topped it with grilled chicken for a quick weeknight dinner
this is a great little side dish for a bbq and much healthier than the typical mayobased macaroni salad olive oil is packed with monounsaturated good fats and can help to achieve balanced cholesterol levels i usually serve this at room temperature but ive also served it hot and topped it with grilled chicken for a quick weeknight dinner
1

this is a great little side dish for a bbq and much healthier than the typical mayobased macaroni salad olive oil is packed with monounsaturated good fats and can help to achieve balanced cholesterol levels i usually serve this at room temperature but ive also served it hot and topped it with grilled chicken for a qui

In [ ]:
df_final2 = pd.DataFrame.from_dict(coeff_dict)
df_final2.columns = data["preprocessed_descriptions"].to_list()
df_final2.index = data["preprocessed_descriptions"].to_list()
df_final2

,this is a great little side dish for a bbq and much healthier than the typical mayobased macaroni salad olive oil is packed with monounsaturated good fats and can help to achieve balanced cholesterol levels i usually serve this at room temperature but ive also served it hot and topped it with grilled chicken for a quick weeknight dinner,recipe courtesy aarti sequeira,i had a half of a cup of finely chopped green olives that id did for tapenade and never finished and i wanted to use them up this is what i made and it was pretty tasty,inspired by a dish i enjoy at a restaurant you can add or substitute other veggies you have around or toss in some cooked crumbled bacon or sausage etc,so simple so delicious so inexpensive you can tweak this recipe in so many ways ive added red pepper and onions left the cheese off for those crazy people that dont like cheese switched up spices its endless
this is a great little side dish for a bbq and much healthier than the typical mayobased macaroni salad olive oil is packed with monounsaturated good fats and can help to achieve balanced cholesterol levels i usually serve this at room temperature but ive also served it hot and topped it with grilled chicken for a quick weeknight dinner,1.000000,0.000000,0.0,0.033553,0.024252
recipe courtesy aarti sequeira,0.000000,1.000000,0.0,0.000000,0.072136
i had a half of a cup of finely chopped green olives that id did for tapenade and never finished and i wanted to use them up this is what i made and it was pretty tasty,0.000000,0.000000,1.0,0.000000,0.000000
inspired by a dish i enjoy at a restaurant you can add or substitute other veggies you have around or toss in some cooked crumbled bacon or sausage etc,0.033553,0.000000,0.0,1.000000,0.000000
so simple so delicious so inexpensive you can tweak this recipe in so many ways ive added red pepper and onions left the cheese off for those crazy people that dont like cheese switched up spices its endless,0.024252,0.072136,0.0,0.000000,1.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [ ]:
print(f"\n{max_pair[0]}\n\n{max_pair[1]}\n\n{max_result}")


recipe courtesy aarti sequeira

so simple so delicious so inexpensive you can tweak this recipe in so many ways ive added red pepper and onions left the cheese off for those crazy people that dont like cheese switched up spices  its endless

0.07213629016660783


In [ ]:
set(max_pair[0].split(" ")) & set(max_pair[1].split(" "))

{'recipe'}